## Loading source files

In [1]:
library(glue)
library(rjson)

## Set iteration times and random seed

In [2]:
set.seed(2020)
iter <- 100

## Check selected $T_0$

In [3]:
T0 <- fromJSON(file = "src/senario_selection.json")$T0
print(glue('Current T0 = {T0}'))

Current T0 = 24


## Parameters

In [4]:
J <- 25  # Control units
T <- 30  # Total periods
tau <- 20  # True treatment effect

## Data Generating Process

In [5]:
for(j in 1:iter){
  print(glue("{j}th dgp starts"))
  flush.console()
  
  # Noisy.
  eps <- matrix(rnorm(J * T, 0, 0.5), nrow = J, ncol = T)
  eps_t <- matrix(rt(J * T, 5), nrow = J, ncol = T)  # t-distributed noisy in Senario D
  eps_treat <- rnorm(T, 0, 1)  # Noisy for treatment unit
  
  # Common trend.
  ct_coef <- 0.25
  ct <- ct_coef * (1:T)
  ct <- matrix(rep(ct, times=J), nrow = J, ncol = T, byrow=TRUE)
  
  # Unit-specific component.
  fixed <- floor(runif(J, 1, 50))
  unit_flat <- 1
  beta_flat <- matrix(rep(unit_flat, times=J), nrow = J, ncol = T, byrow=TRUE)
  for(i in 1:nrow(beta_flat)){
    beta_flat[i,] <- fixed[i] * beta_flat[i,]
  }
  
  # Seasonality.
  eps_sea <- matrix(rnorm(J * T, 0, 0.5), nrow = J, ncol = T)
  sea <- matrix(rep(10 * sin(0.25*(1:T)), times=J), nrow = J, ncol = T, byrow = TRUE) + eps_sea
  #plot(x=1:T, y=sea[10,])
  
  # Treatment unit generated by the first two control units.
  ome_1 <- 0.7
  ome_2 <- 0.3
  omega <- c(ome_1, ome_2, rep(0, times = J - 2))
  
  ## Senario A ##
  
  # Control units
  y_control_A <- beta_flat + ct + eps
  y_control_A <- as.data.frame(y_control_A)
  colnames(y_control_A) <- 1:T
  
  # Generate dataset.
  y_untreat_A_full <- omega %*% as.matrix(y_control_A) + eps_treat
  y_untreat_A <- y_untreat_A_full[1:T0]
  y_treat_A <- y_untreat_A_full[(T0+1):T] + rep(tau, times = T - T0)
  y_treat_A_full <- c(y_untreat_A, y_treat_A)
  
  # Data output.
  file_control = glue('sim_data_control_{j}')
  file_treat = glue('sim_data_treat_{j}')
  file_untreat = glue('sim_data_untreat_{j}')
  
  write.table(y_control_A, file=glue('src/SenarioA/control/{file_control}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(y_treat_A_full, file=glue('src/SenarioA/treat/{file_treat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(as.numeric(y_untreat_A_full), file=glue('src/SenarioA/untreat/{file_untreat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  
  ## Senario B ##
  
  # Control units
  y_control_B <- beta_flat + ct + sea + eps
  y_control_B <- as.data.frame(y_control_B)
  colnames(y_control_B) <- 1:T
  
  # Generate dataset.
  y_untreat_B_full <- omega %*% as.matrix(y_control_B) + eps_treat
  y_untreat_B <- y_untreat_B_full[1:T0]
  y_treat_B <- y_untreat_B_full[(T0+1):T] + rep(tau, times = T - T0)
  y_treat_B_full <- c(y_untreat_B, y_treat_B)
  
  # Data output.
  file_control = glue('sim_data_control_{j}')
  file_treat = glue('sim_data_treat_{j}')
  file_untreat = glue('sim_data_untreat_{j}')
  
  write.table(y_control_B, file=glue('src/SenarioB/control/{file_control}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(y_treat_B_full, file=glue('src/SenarioB/treat/{file_treat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(as.numeric(y_untreat_B_full), file=glue('src/SenarioB/untreat/{file_untreat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  
  ## Senario C ##
  
  # Add time-variant trends to unit-specific components.
  time_trend <- 0.1
  flat_trend <- 1.5
  
  beta_flat_first_two <- matrix(0, ncol=T, nrow=2)
  for(col in 1:T){
    variant_trend <- time_trend * (1:T)
    beta_flat_first_two[,col] <- beta_flat[1:2,][,col] * variant_trend[col]
  }
  
  beta_flat_rest <- beta_flat[3:J,] * flat_trend
  beta_flat_C <- rbind(beta_flat_first_two, beta_flat_rest)
  
  # New seasonality
  sea_C <- matrix(rep(10 * sin(0.1*(1:T)), times=J), nrow = J, ncol = T, byrow = TRUE) + eps_sea
  #plot(x=1:T, y=sea[10,])

  # Control units
  y_control_C <- beta_flat_C + ct + sea_C + eps
  y_control_C <- as.data.frame(y_control_C)
  colnames(y_control_C) <- 1:T
  
  # Generate dataset.
  y_untreat_C_full <- omega %*% as.matrix(y_control_C) + eps_treat
  y_untreat_C <- y_untreat_C_full[1:T0]
  y_treat_C <- y_untreat_C_full[(T0+1):T] + rep(tau, times = T - T0)
  y_treat_C_full <- c(y_untreat_C, y_treat_C)
  
  # Data output.
  file_control = glue('sim_data_control_{j}')
  file_treat = glue('sim_data_treat_{j}')
  file_untreat = glue('sim_data_untreat_{j}')
  
  write.table(y_control_C, file=glue('src/SenarioC/control/{file_control}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(y_treat_C_full, file=glue('src/SenarioC/treat/{file_treat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(as.numeric(y_untreat_C_full), file=glue('src/SenarioC/untreat/{file_untreat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  
  ## Senario D ##
  
  # Control units
  y_control_D <- beta_flat + ct + eps_t
  y_control_D <- as.data.frame(y_control_D)
  colnames(y_control_D) <- 1:T
  
  # Generate dataset.
  y_untreat_D_full <- omega %*% as.matrix(y_control_D) + eps_treat
  y_untreat_D <- y_untreat_D_full[1:T0]
  y_treat_D <- y_untreat_D_full[(T0+1):T] + rep(tau, times = T - T0)
  y_treat_D_full <- c(y_untreat_D, y_treat_D)
  
  # Data output.
  file_control = glue('sim_data_control_{j}')
  file_treat = glue('sim_data_treat_{j}')
  file_untreat = glue('sim_data_untreat_{j}')
  
  write.table(y_control_D, file=glue('src/SenarioD/control/{file_control}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(y_treat_D_full, file=glue('src/SenarioD/treat/{file_treat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  write.table(as.numeric(y_untreat_D_full), file=glue('src/SenarioD/untreat/{file_untreat}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
  
}

1th dgp starts
2th dgp starts
3th dgp starts
4th dgp starts
5th dgp starts
6th dgp starts
7th dgp starts
8th dgp starts
9th dgp starts
10th dgp starts
11th dgp starts
12th dgp starts
13th dgp starts
14th dgp starts
15th dgp starts
16th dgp starts
17th dgp starts
18th dgp starts
19th dgp starts
20th dgp starts
21th dgp starts
22th dgp starts
23th dgp starts
24th dgp starts
25th dgp starts
26th dgp starts
27th dgp starts
28th dgp starts
29th dgp starts
30th dgp starts
31th dgp starts
32th dgp starts
33th dgp starts
34th dgp starts
35th dgp starts
36th dgp starts
37th dgp starts
38th dgp starts
39th dgp starts
40th dgp starts
41th dgp starts
42th dgp starts
43th dgp starts
44th dgp starts
45th dgp starts
46th dgp starts
47th dgp starts
48th dgp starts
49th dgp starts
50th dgp starts
51th dgp starts
52th dgp starts
53th dgp starts
54th dgp starts
55th dgp starts
56th dgp starts
57th dgp starts
58th dgp starts
59th dgp starts
60th dgp starts
61th dgp starts
62th dgp starts
63th dgp starts
6

Click here to go to [02 Matrix completion without cross-validation](02%20Matrix%20completion%20without%20cross-validation.ipynb)